<a href="https://colab.research.google.com/github/CamelGoong/NLP/blob/main/Chapter2_Preparing_NLP_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#01. Tensorflow

더미 데이터를 활용한 감정 분석 모델링(Sentiment Analysis)

>Method1. Sequential API

In [3]:
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples) # 위에서 tokenizer한 samples들을 sequences로 변환

word_index = tokenizer.word_index

In [5]:
# 모델 구축에 필요한 변수 정의
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

In [9]:
import tensorflow.keras.layers as layers
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length = 4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)))
model.add(layers.Dense(hidden_dimension, activation = 'relu'))
model.add(layers.Dense(output_dimension, activation = 'sigmoid'))

In [10]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [11]:
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 6ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6740 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6562 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6372 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6179 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5942 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5686 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5361 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5001 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4625 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

>Method2. Functional API

In [15]:
inputs = layers. Input(shape = (4,)) # 먼저 Input 모듈을 정의해야함.
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis = 1)
hidden_layer = layers.Dense(hidden_dimension, activation = 'relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation = 'sigmoid')(hidden_layer)
model = tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 4ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6760 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6616 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6495 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6319 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6137 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5929 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5692 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5385 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 0.5039 - accuracy: 1.0000
Epoch 11/100
3/3 [==========

>Method3. Custom Model

In [17]:
class CustomModel(tf.keras.Model):
  def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
    super(CustomModel, self).__init__(name = 'my_model')
    self.embedding = layers.Embedding(vocab_size, embed_dimension)
    self.dense_layer = layers.Dense(hidden_dimension, activation = 'relu')
    self.output_layer = layers.Dense(output_dimension, activation = 'sigmoid')

  def call(self, inputs):
    x = self.embedding(inputs)
    x = tf.reduce_mean(x, axis = 1)
    x = self.dense_layer(x)
    x = self.output_layer(x)
    return x

model = CustomModel(vocab_size = vocab_size, embed_dimension = emb_size, hidden_dimension = hidden_dimension, output_dimension = output_dimension)
model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 1s 5ms/step - loss: 0.6952 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6745 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6596 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6441 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6263 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6053 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5810 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.5528 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5211 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4839 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

#02. Scikit-learn